### Import dependences: OpenCV, skimage, numpy, imutils ( Core function is based on opencv and skimage)

In [3]:
import time
import os 
import cv2
import numpy as np
from imutils import paths
from skimage import util
from skimage.segmentation import clear_border
from skimage.measure import label, regionprops

In [4]:
def ComputeMask(img):
    if np.mean(img)>70:
        img=util.invert(img)
        
    # convert to black and white
    ret,mask= cv2.threshold(img,0,1,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    
    # Range filter
    kernel = np.ones((15,15),np.uint8)
    dil=cv2.dilate(img,kernel,iterations = 1)
    ero=cv2.erode(mask,kernel,iterations = 1)
    mask=dil-ero
    ret,mask= cv2.threshold(mask,0,1,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    
    # dilate with horizontal and vertical lines of different size
    for i in range(20,30):
        kernel = np.ones((i,5),np.uint8)
        mask = cv2.dilate(mask,kernel,iterations = 1)    
        
        kernel = np.ones((5,i),np.uint8)
        mask = cv2.dilate(mask,kernel,iterations = 1)
        
        ret,mask= cv2.threshold(mask,0,1,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        
        kernel = np.ones((7,7),np.uint8)
        mask=cv2.erode(mask,kernel,iterations = 2)
        
        
    # invert the mask
    mask=(1-mask)
    
    # Cleaning of the mask
    if np.count_nonzero(clear_border(mask))>0:
        mask = clear_border(mask)
    else:
        s1,s2=mask.shape
        border=mask*0
        border[0:20,:]=mask[0:20,:]
        border[s1-20:s1,:]=mask[s1-20:s1,:]
        border[:,0:20]=mask[:,0:20]
        border[:,s2-20:s2]=mask[:,s2-20:s2]
        mask=mask-border
        
    kernel = np.ones((5,5),np.uint8)
    mask=cv2.erode(mask,kernel,iterations = 10)
    ret,mask= cv2.threshold(mask,0,1,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    return mask


# from the mask compute the bounding box for the mask
def ComputeBoxDim(mask):
    label_image = label(mask)
    max_area=0
    for region in regionprops(label_image):
        if region.area >= max_area:
            minr, minc, maxr, maxc = region.bbox
            max_area=region.area
    return minr, minc, maxr, maxc

# from the bounding box coordinates crop the image of a fixed size (600,1000).
def CropImage(img,minr, minc, maxr, maxc):
    CropImg = img[minr:maxr,  minc:maxc]
    s1,s2=CropImg.shape
    if s1>600 and s2>1000:
        s1=np.uint16(s1/2)
        s2=np.uint16(s2/2)
        CropImg = CropImg[s1-300:s1+300,  s2-500:s2+500]
    return CropImg


def AutomaticBoxCrop(InputPath,DestPath):
    # load all the images in the ImageList
    ImageList = list(paths.list_images(InputPath))

    # Create the destination folder if is does not exist
    if not os.path.exists(DestPath):
        os.makedirs(DestPath)


    # For each image compute the crop image and save it.
    for ImageFile in ImageList:
        img = cv2.imread(ImageFile,0)
        mask=ComputeMask(img)
        minr, minc, maxr, maxc=ComputeBoxDim(mask)
        CropImg=CropImage(img,minr, minc, maxr, maxc)
        FilePath, FileName = os.path.split(ImageFile) 
        DestFile=DestPath+FileName
        print(DestFile)
        cv2.imwrite(DestFile,CropImg)
    



In [5]:
# Inputpath and destination path
InputPath='/home/sumukh/Documents/Upwork/BoxSegmentation/SBM1/SB'
DestPath ='/home/sumukh/Documents/Upwork/BoxSegmentation/SBM1/SB'+'_Croped/'



/home/sumukh/Documents/Upwork/BoxSegmentation/SBM1/SB_Croped/IMG_1102_A2.JPG
/home/sumukh/Documents/Upwork/BoxSegmentation/SBM1/SB_Croped/IMG_1100_A1.JPG
/home/sumukh/Documents/Upwork/BoxSegmentation/SBM1/SB_Croped/IMG_1096_A2.JPG
/home/sumukh/Documents/Upwork/BoxSegmentation/SBM1/SB_Croped/IMG_1102_A1.JPG
/home/sumukh/Documents/Upwork/BoxSegmentation/SBM1/SB_Croped/IMG_1096_A0.JPG
/home/sumukh/Documents/Upwork/BoxSegmentation/SBM1/SB_Croped/IMG_1106_A2.JPG
/home/sumukh/Documents/Upwork/BoxSegmentation/SBM1/SB_Croped/IMG_1104_A2.JPG
/home/sumukh/Documents/Upwork/BoxSegmentation/SBM1/SB_Croped/IMG_1101_A1.JPG
/home/sumukh/Documents/Upwork/BoxSegmentation/SBM1/SB_Croped/IMG_1099_A0.JPG
/home/sumukh/Documents/Upwork/BoxSegmentation/SBM1/SB_Croped/IMG_1100_A0.JPG
/home/sumukh/Documents/Upwork/BoxSegmentation/SBM1/SB_Croped/IMG_1102_A0.JPG
/home/sumukh/Documents/Upwork/BoxSegmentation/SBM1/SB_Croped/IMG_1094_A2.JPG
/home/sumukh/Documents/Upwork/BoxSegmentation/SBM1/SB_Croped/IMG_1100_A2.JPG